Validate a user object with
1. name: non-empty string
2. age: integer > 0
3. email: must contain @
4. is_active: boolean (default: True)

In [ ]:
class User:
    def __init__(self, name, age, email, is_active=True):
        self.name = name
        self.age = age
        self.email = email
        self.is_active = is_active
        self.validate()

    def validate(self):
        if not isinstance(self.name, str) or not self.name.strip():
            raise ValueError("Name must be a non-empty string")

        if not isinstance(self.age, int) or self.age <= 0:
            raise ValueError("Age must be a positive integer")

        if not isinstance(self.email, str) or "@" not in self.email:
            raise ValueError("Invalid email format")

        if not isinstance(self.is_active, bool):
            raise ValueError("is_active must be a boolean")

    def to_dict(self):
        return {
            "name": self.name,
            "age": self.age,
            "email": self.email,
            "is_active": self.is_active,
        }

# Usage
try:
    user = User(name="Jafer", age=25, email="jafer@example.com")
    print(user.to_dict())
except Exception as e:
    print(e)


{'name': 'Jafer', 'age': 25, 'email': 'jafer@example.com', 'is_active': True}


In [ ]:
from pydantic import BaseModel, Field, field_validator

class User(BaseModel):
    name: str = Field(..., min_length=1)
    age: int = Field(..., gt=0)
    email: str
    is_active: bool = True
    pancard: bool = False

    @field_validator("email")
    def email_must_be_valid(cls, v):
        if "@" not in v:
            raise ValueError("Invalid email format")
        return v

# Usage
try:
    user = User(name="Jafer", age=25, email="jafer@example.com")
    print(user.model_dump())
except Exception as e:
    print(e)


{'name': 'Jafer', 'age': 25, 'email': 'jafer@example.com', 'is_active': True}


In [ ]:
def add(num: int):
    print(num)

add("5")

5


In [ ]:
# Basic User Model

In [ ]:
class User(BaseModel):
    id: int
    name: str
    is_active: bool = True

user = User(id='1', name='Jafer', is_active='1')  # str '1' will be coerced to int
print(user.name)
print(user.model_dump())

Jafer
{'id': 1, 'name': 'Jafer', 'is_active': True}


In [ ]:
|# Field Constraints

In [ ]:
from pydantic import BaseModel, Field
class Product(BaseModel):
    name: str = Field(..., min_length=3, max_length=50)
    price: float = Field(..., gt=0, description="Price must be greater than 0")

try:
    product = Product(name='Pen', price=10.5)
    print(product.model_dump())
    print(product.model_dump_json())
except Exception as ex:
    print(ex)

{'name': 'Pen', 'price': 10.5}
{"name":"Pen","price":10.5}


In [ ]:
# Optional Field

In [ ]:
from pydantic import BaseModel
from typing import Optional

class Student(BaseModel):
    name: str
    email: Optional[str] = None

student = Student(name="Alice")
print(student.email)

None


In [ ]:
# Nested Model

In [ ]:
class Address(BaseModel):
    city: str
    pincode: int

class UserWithAddress(BaseModel):
    id: int
    name: str
    address: Address

user_nested = UserWithAddress(id=1, name="Jafer", address={'city': 'Chennai', 'pincode': '600001'})

wijay = UserWithAddress(id=2, name="Wijay", address=user_nested.address)

print(wijay.address)
# print(user_nested.address.city)

city='Chennai' pincode=600001


In [ ]:
user_nested.name

'Jafer'

In [ ]:
user_nested.address

Address(city='Chennai', pincode=600001)

In [ ]:
a

In [ ]:
# Field Validator

In [ ]:
from pydantic import BaseModel, field_validator

class Signup(BaseModel):
    username: str
    password: str

    @field_validator('password')
    def strong_password(cls, v):
        if len(v) < 6:
            raise ValueError('Password too short')
        return v

signup = Signup(username="jafer", password="secu")


ValidationError: 1 validation error for Signup
password
  Value error, Password too short [type=value_error, input_value='secu', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
# Model Validator

In [ ]:
from pydantic import BaseModel, model_validator, Field
class RangeModel(BaseModel):
    start: int = Field(..., gt=12, description="start number")
    end: int

    @model_validator(mode='before')
    def check_range(cls, values):
        start, end = values["start"], values["end"]
        if start > end:
            raise ValueError("start must be less than end")
        return values

range_model = RangeModel(start=11, end=10)


ValidationError: 1 validation error for RangeModel
  Value error, start must be less than end [type=value_error, input_value={'start': 11, 'end': 10}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
# Alias Usage

In [ ]:
class UserAlias(BaseModel):
    names: str = Field(..., alias='full_name')

    class Config:
        validate_by_name = True

user_alias = UserAlias(names="Jafer")
print(user_alias.names)

Jafer


In [ ]:
# Extra Field Handling

In [ ]:
class StudentResponse(BaseModel):
    id: int
    name: str


    class Config:
        extra = "ignore"

extra_user = Todo(id=1, name="Jafer", email="jafer@example.com")
print(extra_user.model_dump())

ValidationError: 1 validation error for Todo
email
  Extra inputs are not permitted [type=extra_forbidden, input_value='jafer@example.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [ ]:
# Inheritance Example

In [ ]:
class BaseUser(BaseModel):
    id: int
    name: str

class AdminUser(BaseUser):
    access_level: int

admin = AdminUser(id=1, name="Syed", access_level=10)
print(admin.model_dump())

{'id': 1, 'name': 'Syed', 'access_level': 10}


In [ ]:
# Environment Variables with BaseSettings

In [ ]:
! pip install pydantic-settings


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00


In [ ]:
from pydantic_settings import BaseSettings
class Settings(BaseSettings):
    app_name: str = "MyApp"
    debug: bool = False

    class Config:
        env_file = ".env"

settings = Settings()
print(settings.app_name, settings.debug)

MyApp False


In [ ]:
# Parsing Raw JSON

In [ ]:
class User(BaseModel):
    id: int
    name: str
    is_active: bool = True

raw_json = '{"id": 123, "name": "Syed", "age": 27}'
user_from_json = User.model_validate_json(raw_json)

print(user_from_json.model_dump())

{'id': 123, 'name': 'Syed', 'is_active': True, 'age': 27}


In [ ]:
# Parsing from Dictionary

In [ ]:
user_dict = {"id": 2, "name": "Asha"}
user_obj = User.model_validate(user_dict)
print(user_obj)

id=2 name='Asha' is_active=True


In [ ]:
# List of Items

In [ ]:
from pydantic import BaseModel
from typing import List

class Item(BaseModel):
    name: str
    quantity: int

class Cart(BaseModel):
    items: List[Item]

cart = Cart(items=[{"name": "Book", "quantity": 2}, {"name": "Pen", "quantity": 5}])
print(cart.model_dump())

{'items': [{'name': 'Book', 'quantity': 2}, {'name': 'Pen', 'quantity': 5}]}


In [ ]:
# Use Union Types

In [ ]:
from pydantic import BaseModel
from typing import Union

class Data(BaseModel):
    value: Union[int, str]

data1 = Data(value=123)
data2 = Data(value="abc")
print(data1, data2)

value=123 value='abc'


In [ ]:
# Reject Extra Fields Strictly


In [ ]:
class StrictUser(BaseModel):
    id: int
    name: str

    class Config:
        extra = "forbid"

try:
    strict_user = StrictUser(id=1, name="Jafer", email="extra")
except Exception as e:
    print(e)

1 validation error for StrictUser
email
  Extra inputs are not permitted [type=extra_forbidden, input_value='extra', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden


In [ ]:
# Data Parsing and Validation

In [ ]:
from datetime import date

In [ ]:
class Event(BaseModel):
    name: str
    event_date: date

event = Event(name="Launch", event_date="2025-12-31")
print(event)

name='Launch' event_date=datetime.date(2025, 12, 31)


In [ ]:
# Boolean From String

In [ ]:
class Toggle(BaseModel):
    active: bool

toggle = Toggle(active="true")
print(toggle)

active=True


In [ ]:
# Zipcode Validation with Regex

In [ ]:
class ZipCode(BaseModel):
    code: str = Field(..., pattern=r"^\d{6}$")

zipcode = ZipCode(code="600001")
print(zipcode)

code='600001'


In [ ]:
# Field Serialization Control

In [ ]:
class UserWithInclude(BaseModel):
    id: int
    name: str
    email: str

user_inc = UserWithInclude(id=1, name="Jafer", email="j@x.com")
print(user_inc.model_dump(include={"id", "email"}))

{'id': 1, 'email': 'j@x.com'}


In [ ]:
# Validate Multiple Field Combinations

In [ ]:
class FullName(BaseModel):
    first_name: str
    last_name: str

    @model_validator(mode="after")
    def no_duplicate(cls, values):
        if values.first_name == values.last_name:
            raise ValueError("First and last names must differ")
        return values

print(FullName(first_name="Syed", last_name="Jafer"))


first_name='Syed' last_name='Jafer'
